In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simsopt
import simsopt.geo
import simsopt.field
import os
import copy
import sys

import plotly.express as px
import plotly.graph_objects as go
from scipy.io import netcdf_file
import numpy as np


Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified



In [60]:
db_path = "QUASR_db/simsopt_serials/1114"

ID = 1010

def get_path(ID, type="simsopt"):
  fID = ID // 1000 
  if type == "simsopt":
    return f'./QUASR_db/simsopt_serials/{fID:04}/serial{ID:07}.json'
  elif(type == "nml"):
    return f'./QUASR_db/nml/{fID:04}/input{ID:07}'
  elif(type == "bdistrib"):
    return f'./QUASR_db/bdistrib_serials/{fID:04}/bdistrib_out.{ID:07}.nc'
  else:
    raise RuntimeError()

sopt_objs = []
for i in range(1000):
  try:
    sopt_objs.append(simsopt.load(get_path(i)))
  except:
    pass
  # simsopt.geo.plot(np.concatenate(sopt_objs[-1]))
len(sopt_objs)

42

In [61]:
df = pd.DataFrame(sopt_objs, columns=["surfaces", "coils"])
df["lcfs"] = df["surfaces"].map(lambda x: x[-1])
df["AR"] = df["lcfs"].map(lambda x: x.aspect_ratio())
df["volume"] = df["lcfs"].map(lambda x: -x.volume())
df["nfp"] = df["lcfs"].map(lambda x: x.nfp)
df["R1"] = df["lcfs"].map(lambda x: x.minor_radius())

print(len(df))
df.head()

42


,surfaces,coils,lcfs,AR,volume,nfp,R1
0,[SurfaceXYZTensorFourier966],"[Coil1897, Coil1898, Coil1899, Coil1900]",SurfaceXYZTensorFourier966,19.999985,0.049348,2,0.050000
1,"[SurfaceXYZTensorFourier967, SurfaceXYZTensorF...","[Coil1901, Coil1902, Coil1903, Coil1904]",SurfaceXYZTensorFourier968,9.999963,0.197392,2,0.100000
2,"[SurfaceXYZTensorFourier969, SurfaceXYZTensorF...","[Coil1905, Coil1906, Coil1907, Coil1908]",SurfaceXYZTensorFourier971,6.666613,0.444132,2,0.150000
3,"[SurfaceXYZTensorFourier972, SurfaceXYZTensorF...","[Coil1909, Coil1910, Coil1911, Coil1912]",SurfaceXYZTensorFourier977,3.333200,1.776529,2,0.300004
4,"[SurfaceXYZTensorFourier978, SurfaceXYZTensorF...","[Coil1913, Coil1914, Coil1915, Coil1916]",SurfaceXYZTensorFourier984,2.856976,2.418053,2,0.350007


In [4]:
def eval_row(row, plot=False):
  coils = row["coils"]
  nfp = row["nfp"]
  R1 = row["R1"]
  lcfs = row["lcfs"]
  computational_surface = simsopt.geo.SurfaceRZFourier.from_nphi_ntheta(nfp=nfp, nphi=64, ntheta=64, range="half period")

  # Simple torus boundary
  computational_surface.set_rc(1,0,R1*2.0)
  computational_surface.set_zs(1,0,R1*2.0)
  computational_surface.change_resolution(3,4)
  # computational_surface.set_rc(2,1,np.random.rand())
  # computational_surface.set_zs(1,2,np.random.rand())
  normal_computational = computational_surface.normal()
  xyz_computational = computational_surface.gamma()

  # Scaled computational boundary
  scale = 1.5
  lcfs.scale(scale)
  xyz_computational = lcfs.gamma()
  normal_computational = lcfs.normal()
  lcfs.scale(1.0 / scale)

  bs = simsopt.field.BiotSavart(coils)
  bs.set_points_cart(xyz_computational.reshape((-1,3)))
  B = bs.B().reshape(xyz_computational.shape)
  BdotN = np.sum(normal_computational * B, axis=-1)
  BdotN_fft = np.fft.fft2(BdotN)
  
  if plot:
    import plotly.express as px
    # px.imshow(BdotN).show()
    # px.imshow(np.abs(np.fft.fftshift(BdotN_fft))).show()
    # px.imshow(np.real(np.fft.fftshift(BdotN_fft)), title="Real component").show()
    # px.imshow(np.imag(np.fft.fftshift(BdotN_fft)), title="Imag component").show()
    plt.figure(figsize=(16,5))
    plt.subplot(131)
    plt.imshow(BdotN)
    plt.title("BdotN")
    plt.colorbar()
    plt.subplot(132)
    plt.imshow(np.real(np.fft.fftshift(BdotN_fft)))
    plt.title("fft real")
    plt.colorbar()
    plt.subplot(133)
    plt.imshow(np.imag(np.fft.fftshift(BdotN_fft)))
    plt.title("fft imag")
    plt.colorbar()
    # plt.tight_layout()
    plt.show()

  return BdotN_fft

for idx,row in df.iterrows():
  eval_row(row, False)

- Check simple computational boundaries
- Check using a simple scaled up boundary
- Is SVD independent of boundary geometry?

In [5]:
dfid = 3
lcfs = df["lcfs"][dfid]
curves = [coil.curve for coil in df["coils"][dfid]]

# XYZ tensor fourier -> RZ fourier
rzf = lcfs.to_RZFourier()
rzf

SurfaceRZFourier43 (nfp=4, stellsym=True, mpol=10, ntor=10)

In [6]:
import numpy as np
from scipy.spatial.distance import cdist

def minimum_coil_surf_distance(curves, lcfs)->float:
  min_dist = np.inf
  pointcloud1 = lcfs.gamma().reshape((-1, 3))
  for c in curves:
    pointcloud2 = c.gamma()
    min_dist = min(min_dist, np.min(cdist(pointcloud1, pointcloud2)))
    # Equivalent python code:
    # for point in pointcloud2:
    #   min_dist = min(min_dist, np.min(np.linalg.norm(pointcloud1 - point, axis=-1)))
    
  return float(min_dist)


In [57]:
# simsopt.geo.plot([df["lcfs"][dfid]] + df["coils"][dfid], engine="plotly")
(lcfs.minor_radius(), minimum_coil_surf_distance(curves, lcfs))

(0.2500058222924249, 0.3518446936292137)

In [ ]:
def netcdf_from_surface(surface:simsopt.geo.SurfaceRZFourier):
  import os

  filename_out = "wout_surfaces_python_generated.nc"
  filename = "../bdistrib/equilibria/wout_w7x_standardConfig.nc"
  os.system(f'cp {filename} {filename_out}')  

  # Copy the file on disk with a new name, open with r+ and overwrite it.
  with netcdf_file(filename_out, "a", mmap=False) as f:
    print(list(f.variables.keys()))
    print(f.variables["zmns"][()].shape, f.variables["zmns"].units, f.variables["zmns"].dimensions,  f.variables["zmns"][()].dtype)
    print(np.max(f.variables["zmns"][()]))

    # The plasma surface read in by bdistrib is zmns[ns] & rmnc[ns]
    # nfp_vmec = nfp
    # Rmajor_p = R0 

    # implicitly broadcasts the result throughout all flux surfaces
    mpol = int(f.variables["mpol"][()])-1
    ntor = int(f.variables["ntor"][()])
    rzf.change_resolution(mpol, ntor)
    f.variables["rmnc"][:] = surface.rc.flatten()[rzf.ntor:]
    f.variables["zmns"][:] = -surface.zs.flatten()[rzf.ntor:] # Right and left handed coordinates. Flip the signs of sin(theta) to match

    f.variables["Rmajor_p"][()] = surface.major_radius() 
    f.variables["nfp"][()] = surface.nfp 

    # TODO: Net poloidal current profile (bvco), net poloidal current Amperes is computed from this
    # net_poloidal_current_Amperes = (2*pi/mu0) * (1.5*bvco(end) - 0.5*bvco(end-1));
    # f.variables["bvco"][:] = np.zeros(f.variables["lmns"][()].shape)

    # TODO: set lmns component? sinmn component of lambda, half mesh
    # TODO: GMNC component? Both dont seem to have an impact
    # f.variables["gmnc"][:] = np.zeros(f.variables["gmnc"][()].shape)
    # f.variables["lmns"][:] = np.zeros(f.variables["lmns"][()].shape)
    
    # HACK sets the success flag to true so the input reading doesnt fail
    f.variables["ier_flag"][()] = 0
  
  return filename_out.replace("wout_","")

def write_bdistribin(netcdffilename, mpol = 14, ntor = 14, sep_outer=0.1):
  nu = 64
  nv = 256

  sep_middle = sep_outer/2

  bdistribin = f"""&bdistrib
    transfer_matrix_option = 1

    nu_plasma={nu}
    nu_middle={nu}
    nu_outer ={nu}

    nv_plasma={nv}
    nv_middle={nv}
    nv_outer ={nv}

    ! This run is not well resolved at this resolution, but it is enough for testing.
    mpol_plasma = {mpol}
    mpol_middle = {mpol}
    mpol_outer  = {mpol}

    ntor_plasma = {ntor}
    ntor_middle = {ntor}
    ntor_outer  = {ntor}

    geometry_option_plasma = 2
    wout_filename='{netcdffilename}'

    geometry_option_middle=2
    separation_middle={sep_middle}

    geometry_option_outer=2
    separation_outer={sep_outer}

    pseudoinverse_thresholds = 1e-12

    n_singular_vectors_to_save = 16
  /
  """
  filename_out = "bdistrib_in.python_generated"
  with open(filename_out, "w") as f:
    f.write(bdistribin)
  return filename_out


import subprocess
subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(rzf), 
                                                                sep_outer=minimum_coil_surf_distance(curves, rzf))])

# rzf.plot()
import bdistribPlot
bdistribPlot.main("bdistrib_out.python_generated.nc")

In [ ]:
def get_LgradB():
  from simsopt.util import MpiPartition
  from simsopt.mhd import Vmec
  
  mpi = MpiPartition(ngroups=3)

  equil = Vmec('input.2DOF_vmecOnly_targetIotaAndVolume', mpi)

In [11]:
for ID in range(1000, 2000):
  simsopt_path = get_path(ID)
  if os.path.exists(simsopt_path):
    soptobj = simsopt.load(simsopt_path)

    lcfs = soptobj[0][-1]
    # XYZ tensor fourier -> RZ fourier
    rzf = lcfs.to_RZFourier()
    curves = [coil.curve for coil in soptobj[1]]
    
    bdistrib_out_path = get_path(ID, "bdistrib")
    subprocess.check_call(["mkdir", "-p", os.path.dirname(bdistrib_out_path)])
    subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(rzf), 
                                                                    sep_outer=minimum_coil_surf_distance(curves, rzf))])
    # Move results to correct directory
    subprocess.check_call(["mv", "bdistrib_out.python_generated.nc", bdistrib_out_path, "-u"])
  else:
    print("Skipping", simsopt_path)

['version_', 'input_extension', 'mgrid_file', 'wb', 'wp', 'gamma', 'rmax_surf', 'rmin_surf', 'zmax_surf', 'nfp', 'ns', 'mpol', 'ntor', 'mnmax', 'mnmax_nyq', 'niter', 'itfsq', 'lasym__logical__', 'lrecon__logical__', 'lfreeb__logical__', 'ier_flag', 'aspect', 'betatotal', 'betapol', 'betator', 'betaxis', 'b0', 'rbtor0', 'rbtor', 'signgs', 'IonLarmor', 'volavgB', 'ctor', 'Aminor_p', 'Rmajor_p', 'volume_p', 'nextcur', 'extcur', 'mgrid_mode', 'nobser', 'nobd', 'nbsets', 'xm', 'xn', 'xm_nyq', 'xn_nyq', 'raxis_cc', 'zaxis_cs', 'iotaf', 'presf', 'phi', 'phipf', 'jcuru', 'jcurv', 'iotas', 'mass', 'pres', 'beta_vol', 'buco', 'bvco', 'vp', 'specw', 'phips', 'over_r', 'jdotb', 'bdotgradv', 'DMerc', 'DShear', 'DWell', 'DCurr', 'DGeod', 'equif', 'fsqt', 'wdot', 'curlabel', 'rmnc', 'zmns', 'lmns', 'gmnc', 'bmnc', 'bsubumnc', 'bsubvmnc', 'bsubsmns', 'bsupumnc', 'bsupvmnc']
(99, 288) b'm' ('radius', 'mn_mode') >f8
0.6658179614437796
 This is BDISTRIB.
 Successfully read parameters from bdistrib nameli

In [ ]:
import bdistribPlot

for ID in range(1000, 2000):
  bdistrib_path = get_path(ID, "bdistrib")
  simsopt_path = get_path(ID, "simsopt")
  if os.path.exists(bdistrib_path):
    print(bdistrib_path)
    bdistribPlot.main(bdistrib_path)
    # soptobj = simsopt.load(simsopt_path)
    # simsopt.geo.plot(soptobj[0] + soptobj[1], engine="plotly")

In [54]:
tokamak_surf = simsopt.geo.SurfaceRZFourier(nfp=2, stellsym=False, mpol=5, ntor=5)
tokamak_surf.set_zs(1,0, 0.5)
tokamak_surf.set_rc(1,0, 0.4)
tokamak_surf.set_rs(1,0, 0.1)
tokamak_surf.set_rs(2,0, 0.2)


subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(tokamak_surf), 
                                                                sep_outer=0.2)])

bdistribPlot.main("bdistrib_out.tokamak.nc")

['version_', 'input_extension', 'mgrid_file', 'wb', 'wp', 'gamma', 'rmax_surf', 'rmin_surf', 'zmax_surf', 'nfp', 'ns', 'mpol', 'ntor', 'mnmax', 'mnmax_nyq', 'niter', 'itfsq', 'lasym__logical__', 'lrecon__logical__', 'lfreeb__logical__', 'ier_flag', 'aspect', 'betatotal', 'betapol', 'betator', 'betaxis', 'b0', 'rbtor0', 'rbtor', 'signgs', 'IonLarmor', 'volavgB', 'ctor', 'Aminor_p', 'Rmajor_p', 'volume_p', 'nextcur', 'extcur', 'mgrid_mode', 'nobser', 'nobd', 'nbsets', 'xm', 'xn', 'xm_nyq', 'xn_nyq', 'raxis_cc', 'zaxis_cs', 'iotaf', 'presf', 'phi', 'phipf', 'jcuru', 'jcurv', 'iotas', 'mass', 'pres', 'beta_vol', 'buco', 'bvco', 'vp', 'specw', 'phips', 'over_r', 'jdotb', 'bdotgradv', 'DMerc', 'DShear', 'DWell', 'DCurr', 'DGeod', 'equif', 'fsqt', 'wdot', 'curlabel', 'rmnc', 'zmns', 'lmns', 'gmnc', 'bmnc', 'bsubumnc', 'bsubvmnc', 'bsubsmns', 'bsupumnc', 'bsupvmnc']
(99, 288) b'm' ('radius', 'mn_mode') >f8
0.6658179614437796


ValueError: could not broadcast input array from shape (54,) into shape (99,288)